In [1]:
import gmaps
import pandas as pd
import numpy as np
import json
import seaborn
import requests
import matplotlib.pyplot as plt

# Google developer API key
from api_key import gmap_api

# Access maps with unique API key
gmaps.configure(api_key=gmap_api)

In [2]:
#read in combined csvfile
va_agencies_df = pd.read_csv("../data/cleandata/va_combined_2005-2015.csv")
va_louisa_df = va_agencies_df.loc[va_agencies_df["report_year"] == 2011, :]

#drop records where lat or long is Null
clean_va_agencies_df = va_agencies_df.dropna(subset = ['icpsr_lat', 'icpsr_lng'])

#limit  data to year of natural disaster
louisa_agencies_pd = clean_va_agencies_df.loc[clean_va_agencies_df["report_year"] == 2011, :]


FileNotFoundError: File b'../data/cleandata/va_combined_2005-2015.csv' does not exist

In [ ]:
# show only participating agencies
part_louisa_agencies_df = louisa_agencies_pd.loc[louisa_agencies_pd["nibrs_months_reported"] == 12, :]
part_louisa_agencies_df = pd.DataFrame(part_louisa_agencies_df)


In [ ]:
# Store 'Lat' and 'Lng' into  locations 
part_va_locations = part_louisa_agencies_df[["icpsr_lat", "icpsr_lng"]].astype(float)
part_va_locations.head()

In [ ]:
# Store 'Lat' and 'Lng' into  locations 
va_locations = louisa_agencies_pd[["icpsr_lat", "icpsr_lng"]].astype(float)
va_locations.head()

In [ ]:
#get totals for reports
all_years = va_agencies_df.count()
all = all_years.max()

clean_all_years = clean_va_agencies_df.count()
clean = clean_all_years.max()

va_agencies = va_louisa_df.count()
va = va_agencies.max()

clean_louisa = louisa_agencies_pd.count()
louisa = clean_louisa.max()

part_louisa = part_louisa_agencies_df.count()
part = part_louisa.max()

coord_part = part_va_locations.count()
part_loc = coord_part.max()

coord_va = va_locations.count()
va_loc = coord_va.max()

totals = {'va 2005 - 2015': all, 'Cleaned va all years': clean, 'All va 2011': va, 'Cleaned va 2011': louisa, 
          'Cleaned va Coordinates': va_loc, 'Particpating': part, 'Part Locations': part_loc}
totals

In [ ]:
# Customize the size of the figure
part_figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
part_fig = gmaps.figure(layout=part_figure_layout)

In [ ]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(va_locations,
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

# Adjust heat_layer setting to help with heatmap dissipating on zoom
heat_layer.dissipating = False
heat_layer.max_intensity = 100
heat_layer.point_radius = 2

fig.add_layer(heat_layer)

fig

In [ ]:
# Assign the marker layer to a variable
markers = gmaps.marker_layer(part_va_locations)
# Add the layer to the map
fig.add_layer(markers)
fig